# Draw pairs of mass and redshift according to a mass function (computed with either CCL or NumCosmo)

In [ ]:
import sys
import numpy as np
import pyccl as ccl
import matplotlib.pyplot as plt
import scipy 
%matplotlib inline

In [ ]:
# NumCosmo imports
try:
  import gi
  gi.require_version('NumCosmo', '1.0')
  gi.require_version('NumCosmoMath', '1.0')
except:
  pass

from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

## Mass function - CCL implementation

In [ ]:
cosmo = ccl.Cosmology(Omega_c=0.26, Omega_b=0.04,
                      h=0.7, sigma8=0.8, n_s=0.96, Neff=3.04)
hmd_200c = ccl.halos.MassDef(200, 'critical')

def tinker08_ccl(logm, z):
    mass = 10**(logm)
    hmf_200c = ccl.halos.MassFuncTinker08(cosmo, mass_def=hmd_200c)
    nm = hmf_200c.get_mass_function(cosmo, mass, 1./(1+z))
    return nm # dn/dlog10M

def dV_over_dOmega_dz(z):
    a = 1./(1. + z)
    da = ccl.background.angular_diameter_distance(cosmo, a) 
    E = ccl.background.h_over_h0(cosmo, a)
    return ((1.+z)**2)*(da**2)*ccl.physical_constants.CLIGHT_HMPC/cosmo['h']/E 

def pdf_tinker08_ccl(logm,z):
    return tinker08_ccl(logm, z)*dV_over_dOmega_dz(z)


## Mass function - Numcosmo implementation

In [ ]:
Ncm.cfg_init ()
#cosmo_nc = Nc.HICosmoDEXcdm()
cosmo_nc = Nc.HICosmo.new_from_name(
    Nc.HICosmo, "NcHICosmoDECpl{'massnu-length':<1>}")
cosmo_nc.omega_x2omega_k()
cosmo_nc.param_set_by_name("w0", -1.0)
cosmo_nc.param_set_by_name("w1", 0.0)
cosmo_nc.param_set_by_name("Tgamma0", 2.725)
cosmo_nc.param_set_by_name("massnu_0", 0.0)
cosmo_nc.param_set_by_name("H0", 70.)
cosmo_nc.param_set_by_name("Omegab", 0.04)
cosmo_nc.param_set_by_name("Omegac", 0.26)
cosmo_nc.param_set_by_name("Omegak", 0.0)

# ENnu = 3.046 - 3.0 * \
#     cosmo_nc.E2Press_mnu(1.0e10) / (cosmo_nc.E2Omega_g(1.0e10)
#                                  * (7.0/8.0*(4.0/11.0)**(4.0/3.0)))

ENnu = 3.046
cosmo_nc.param_set_by_name("ENnu", ENnu)
reion = Nc.HIReionCamb.new () 
prim = Nc.HIPrimPowerLaw.new () 

cosmo_nc.add_submodel (reion)
cosmo_nc.add_submodel (prim)

dist = Nc.Distance.new (2.0)
dist.prepare_if_needed(cosmo_nc)
tf = Nc.TransferFunc.new_from_name ("NcTransferFuncEH")

psml = Nc.PowspecMLTransfer.new (tf)
psml.require_kmin (1.0e-6)
psml.require_kmax (1.0e3)

psf = Ncm.PowspecFilter.new (psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0 ()

prim.props.n_SA = 0.96

old_amplitude = np.exp (prim.props.ln10e10ASA)
prim.props.ln10e10ASA = np.log ((0.8 / cosmo_nc.sigma8(psf))**2 * old_amplitude)
print(0.8, cosmo_nc.sigma8(psf))

mulf = Nc.MultiplicityFuncTinker.new ()
mulf.set_mdef (Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta (200.0)
mf = Nc.HaloMassFunction.new (dist, psf, mulf)
#
# New mass function object using the objects defined above.
#
def tinker08_nc(logm, z):
    lnm = logm * np.log(10.) # convert log10(M) to ln(M) 
    res = mf.dn_dlnM(cosmo_nc, lnm, z)
    return res * np.log(10.) # convert dn/dlnM to dn/dlog10M

def pdf_tinker08_nc(logm, z):
    return tinker08_nc(logm, z)*mf.dv_dzdomega(cosmo_nc, z)

## Compare the CCL and NC implementations

### Volume element

In [ ]:
z_arr = np.linspace(0.01, 2, 100)
v_nc = [mf.dv_dzdomega(cosmo_nc, z) for z in z_arr]
v_ccl = dV_over_dOmega_dz(z_arr)
plt.plot(z_arr, np.abs(v_nc/v_ccl -1)*100);
plt.xlabel('Redshift', size=14);
plt.ylabel('Relative difference [%]', size=14);

### Mass function

The CCL and Numcosmo implementation gives different results for the Tinker08 HMF, especially at high z. Need to understand why. Might be that the CCL and NC cosmologies defined above are not the same (neutrinos?)

In [ ]:
logm_arr=np.linspace(14.,15)
z_arr = np.linspace(0.01, 1.7, 2)

t08_vec_ccl=np.vectorize(tinker08_ccl)
t08_vec_nc=np.vectorize(tinker08_nc)
for z in z_arr:
    res_ccl = t08_vec_ccl(logm_arr,z)
    res_nc = t08_vec_nc(logm_arr,z)
    plt.plot(logm_arr, res_ccl, label=f'z={z}, CCL');
    plt.plot(logm_arr, res_nc, label=f'z={z}, NC');
    plt.yscale('log');
    plt.xlabel('log10(M200,c)', size=14);
    plt.ylabel('dn/d$\log_{10}$M', size=14);
plt.legend();

In [ ]:
z_arr = np.linspace(0.01, 1.7, 100)
logm = 15.
res_ccl = t08_vec_ccl(logm,z_arr)
res_nc = t08_vec_nc(logm,z_arr)
plt.plot(z_arr, np.abs(res_ccl/res_nc -1)*100);
plt.xlabel("Redshift", size=14);
plt.ylabel("Relative difference [%]", size=14);

## Acceptance-rejection method to sample (M,z) from the mass function

In [ ]:
def bivariate_draw(pdf, N=1000,logm_min=14.,logm_max=15., zmin=0.01, zmax=1, Ngrid_m=30, Ngrid_z=30):  
    """  
    Uses the rejection method for generating random numbers derived from an arbitrary   
    probability distribution. 

    Parameters
    ----------
    pdf : func
      2d distribution function to sample
    N : int
      number of points to generate  
    log_min,logm_max : float 
      log10 mass range  
    zmin,zmax : float 
      redshift range  

    Returns:   
    --------
    ran_logm : list
        accepted logm values  
    ran_z : list
        accepted redshift values
    acceptance : float
        acceptance ratio of the method
    """  

    # maximum value of the pdf over the mass and redshift space.
    # the pdf is not monotonous in mass and redshift, so we need
    # to find the maximum numerically. Here we scan the space
    # with a regular grid and use the maximum value.
    # Accuracy of the results depends on Ngrid_m and Ngrid_z 
    # This can probably be improved
    
    logM_arr = np.linspace(logm_min, logm_max, Ngrid_m)  
    z_arr = np.logspace(np.log10(zmin), np.log10(zmax), Ngrid_z)  
    p = []
    for logM in logM_arr:
        for z in z_arr:
            p.append(pdf(logM, z))  
    pmax = np.max(p)
    
    # Counters  
    naccept = 0  
    ntrial = 0  

    # Keeps drawing until N points are accepted  
    ran_logm = [] # output list of random numbers  
    ran_z = [] # output list of random numbers  
    while naccept < N:  
        # draw (logm,z) from uniform distribution
        # draw p from uniform distribution
        logm = np.random.uniform(logm_min,logm_max) # x'  
        z = np.random.uniform(zmin,zmax) # x'  
        p = np.random.uniform(0,pmax) # y'  

        if p < pdf(logm, z):  
            # keep the point
#            print(logm, z, p, pdf(logm, z))
            ran_logm.append(logm)  
            ran_z.append(z)  
            naccept = naccept+1  
        ntrial = ntrial+1  

    ran_logm = np.asarray(ran_logm)  
    ran_z = np.asarray(ran_z)  

    acceptance = float(N/ntrial)
    print(f"acceptance = {acceptance}")
    return ran_logm, ran_z, acceptance

## Draw (M,z) pairs and check consistency with mass function

In [ ]:
# Define the numbers of pairs to draw and the mass and redshift ranges
N = 4000
logm_min = 14.
logm_max = 14.5
zmin = 0.2
zmax = 1.

# Choose between the CCL and NC implementation (NC is faster)
# pdf = pdf_tinker08_nc
pdf = pdf_tinker08_nc

In [ ]:
# Normalisation factor for the pdf to be used later
norm = 1./scipy.integrate.dblquad(pdf, zmin, zmax, lambda x:logm_min, lambda x:logm_max, epsrel=1.e-4)[0]

In [ ]:
# Random draw
ran_logm, ran_z, acceptance = bivariate_draw(pdf, N=N,logm_min=logm_min,logm_max=logm_max, zmin=zmin, zmax=zmax)

In [ ]:
plt.hist2d(ran_logm, ran_z, bins=5);

## Check distribution in mass bins (integrate over the full redshift range)

In [ ]:
hist_m = plt.hist(ran_logm, bins=10);
plt.yscale('log');
plt.xlabel('$\log_{10}$M', size=14);
print(hist_m[1])

In [ ]:
print(zmin, zmax)
predicted_T08_m_quad=[]
for i in np.arange(len(hist_m[0])):
    lmmin = hist_m[1][i]
    lmmax = hist_m[1][i+1]
    N_T08 =  norm * N * scipy.integrate.dblquad(pdf, zmin, zmax, lambda x:lmmin, lambda x:lmmax, epsrel=1.e-4)[0]
    predicted_T08_m_quad.append(N_T08) 


In [ ]:
bin_centers_m=[(hist_m[1][i]+hist_m[1][i+1])/2. for i in np.arange(len(hist_m[0]))]

hist_m = plt.hist(ran_logm, bins=10, color='teal', alpha=0.3, label='drawn masses - integrated over redshift range');
plt.plot(bin_centers_m,np.array(predicted_T08_m_quad), '-o',color='green', label='Expected from theory');
plt.yscale('log');
plt.xlabel('$\log_{10}$M', size=14);
plt.ylabel('Number', size=14);
plt.title(f'{N} clusters drawn in log10M={logm_min, logm_max}, z={zmin, zmax}');
plt.legend();

## Check distribution in redshift bins (integrate over the full mass range)

In [ ]:
hist_z = plt.hist(ran_z, bins=10);
plt.yscale('log');
plt.xlabel('Redshift', size=14);

In [ ]:
print(logm_min, logm_max)
predicted_T08_z_quad=[]
for i in np.arange(len(hist_z[0])):
    zzmin = hist_z[1][i] 
    zzmax = hist_z[1][i+1]
    N_T08 = norm * N * scipy.integrate.dblquad(pdf, zzmin, zzmax, lambda x:logm_min, lambda x:logm_max, epsabs=1.e-4, epsrel=1.e-4)[0]
    predicted_T08_z_quad.append(N_T08) 


In [ ]:
bin_centers_z=[(hist_z[1][i]+hist_z[1][i+1])/2. for i in np.arange(len(hist_z[0]))]

hist_z = plt.hist(ran_z, bins=10, color='orange', alpha=0.3, label='drawn redshift - integrated over mass range');
plt.plot(bin_centers_z,np.array(predicted_T08_z_quad), '-o',color='darkorange', label='Expected from theory');
plt.yscale('log');
plt.xlabel('Redshift', size=14);
plt.ylabel('Number', size=14);
plt.title(f'{N} clusters drawn in log10M={logm_min, logm_max}, z={zmin, zmax}');
plt.legend(loc=4);